In [1]:
import sys
from pathlib import Path

import qlib
import pandas as pd
from qlib.config import REG_CN
from qlib.contrib.model.gbdt import LGBModel
from qlib.contrib.estimator.handler import QLibDataHandlerV1
from qlib.contrib.strategy.strategy import TopkAmountStrategy
from qlib.contrib.evaluate import (
    backtest as normal_backtest,
    long_short_backtest,
    risk_analysis,
)
from qlib.utils import exists_qlib_data

In [2]:
# use default data
# NOTE: need to download data from remote: python scripts/get_data.py qlib_data --target_dir ~/.qlib/qlib_data/cn_data
mount_path = "~/.qlib/qlib_data/cn_data"  # target_dir
if not exists_qlib_data(mount_path):
    print(f"Qlib data is not found in {mount_path}")
    sys.path.append(str(Path(__file__).resolve().parent.parent.joinpath("scripts")))
    from get_data import GetData
    GetData().qlib_data_cn(mount_path)
qlib.init(mount_path=mount_path, region=REG_CN)

[47780:MainThread](2020-09-18 10:50:24,791) INFO - qlib.Initialization - [__init__.py:39] - default_conf: client.
[47780:MainThread](2020-09-18 10:50:24,794) WARNING - qlib.Initialization - [__init__.py:62] - redis connection failed(host=127.0.0.1 port=6379), cache will not be used!
[47780:MainThread](2020-09-18 10:50:24,795) WARNING - qlib.Initialization - [__init__.py:77] - auto_path is False, please make sure /home/v-xiabi/.qlib/qlib_data/cn_data is mounted
[47780:MainThread](2020-09-18 10:50:24,795) INFO - qlib.Initialization - [__init__.py:172] - qlib successfully initialized based on client settings.
[47780:MainThread](2020-09-18 10:50:24,796) INFO - qlib.Initialization - [__init__.py:178] - mount_path=/home/v-xiabi/.qlib/qlib_data/cn_data


In [3]:
MARKET = "csi500"
BENCHMARK = "SH000905"

# train model

In [4]:
###################################
# train model
###################################
DATA_HANDLER_CONFIG = {
    "dropna_label": True,
    "start_date": "2007-01-01",
    "end_date": "2020-08-01",
    "market": MARKET,
}

TRAINER_CONFIG = {
    "train_start_date": "2007-01-01",
    "train_end_date": "2014-12-31",
    "validate_start_date": "2015-01-01",
    "validate_end_date": "2016-12-31",
    "test_start_date": "2017-01-01",
    "test_end_date": "2020-08-01",
}

# use default DataHandler
# custom DataHandler, refer to: TODO: DataHandler api url
x_train, y_train, x_validate, y_validate, x_test, y_test = QLibDataHandlerV1(
    **DATA_HANDLER_CONFIG
).get_split_data(**TRAINER_CONFIG)


MODEL_CONFIG = {
    "loss": "mse",
    "colsample_bytree": 0.8879,
    "learning_rate": 0.0421,
    "subsample": 0.8789,
    "lambda_l1": 205.6999,
    "lambda_l2": 580.9768,
    "max_depth": 8,
    "num_leaves": 210,
    "num_threads": 20,
}
# use default model
# custom Model, refer to: TODO: Model api url
model = LGBModel(**MODEL_CONFIG)
model.fit(x_train, y_train, x_validate, y_validate)
_pred = model.predict(x_test)
_pred = pd.DataFrame(_pred, index=x_test.index, columns=y_test.columns)

# backtest requires pred_score
pred_score = pd.DataFrame(index=_pred.index)
pred_score["score"] = _pred.iloc(axis=1)[0]

# save pred_score to file
pred_score_path = Path("~/tmp/qlib/pred_score.pkl").expanduser()
pred_score_path.parent.mkdir(exist_ok=True, parents=True)
pred_score.to_pickle(pred_score_path)

~/.qlib/qlib_data/cn_data/instruments/csi500.txt


ValueError: instruments not exists for market csi500

# backtest

In [5]:
###################################
# backtest
###################################
STRATEGY_CONFIG = {
    "topk": 50,
    "buffer_margin": 230,
}
BACKTEST_CONFIG = {
    "verbose": False,
    "limit_threshold": 0.095,
    "account": 100000000,
    "benchmark": BENCHMARK,
    "deal_price": "vwap",
    "open_cost": 0.0005,
    "close_cost": 0.0015,
    "min_cost": 5,
    
}

# use default strategy
# custom Strategy, refer to: TODO: Strategy api url
strategy = TopkAmountStrategy(**STRATEGY_CONFIG)
report_normal, positions_normal = normal_backtest(
    pred_score, strategy=strategy, **BACKTEST_CONFIG
)

# long short backtest
long_short_reports = long_short_backtest(pred_score, topk=50)

NameError: name 'pred_score' is not defined

# analyze

In [6]:
###################################
# analyze
# If need a more detailed analysis, refer to: examples/train_and_bakctest.ipynb
###################################
analysis = dict()
analysis["pred_long"] = risk_analysis(long_short_reports["long"])
analysis["pred_short"] = risk_analysis(long_short_reports["short"])
analysis["pred_long_short"] = risk_analysis(long_short_reports["long_short"])
analysis["sub_bench"] = risk_analysis(report_normal["return"] - report_normal["bench"])
analysis["sub_cost"] = risk_analysis(
    report_normal["return"] - report_normal["bench"] - report_normal["cost"]
)
analysis_df = pd.concat(analysis)  # type: pd.DataFrame
print(analysis_df)

NameError: name 'long_short_reports' is not defined

# analyze graphs

In [7]:
from qlib.contrib.report import analysis_model, analysis_position

In [8]:
# get label data
from qlib.data import D
pred_df_dates = pred_score.index.get_level_values(level='datetime')
features_df = D.features(D.instruments(MARKET), ['Ref($close, -1)/$close - 1'], pred_df_dates.min(), pred_df_dates.max())
features_df.columns = ['label']

NameError: name 'pred_score' is not defined

## analysis position

### report

In [9]:
analysis_position.report_graph(report_normal)

NameError: name 'report_normal' is not defined

### score IC

In [10]:
pred_label = pd.concat([features_df, pred_score], axis=1, sort=True).reindex(features_df.index)
analysis_position.score_ic_graph(pred_label)

NameError: name 'features_df' is not defined

### cumulative return

In [11]:
analysis_position.cumulative_return_graph(positions_normal, report_normal, features_df)

NameError: name 'positions_normal' is not defined

### risk analysis

In [12]:
report_long_short_df = pd.concat(long_short_reports, axis=1)
analysis_position.risk_analysis_graph(analysis_df, report_normal, report_long_short_df)

NameError: name 'long_short_reports' is not defined

### rank label

In [13]:
analysis_position.rank_label_graph(positions_normal, features_df, pred_df_dates.min(), pred_df_dates.max())

NameError: name 'positions_normal' is not defined

## analysis model

### model performance

In [14]:
analysis_model.model_performance_graph(pred_label)

NameError: name 'pred_label' is not defined